In [ ]:
import pandas as pd
import numpy as np

file_path = r"C:\Users\japar\OneDrive\Desktop\SQL\Drug Related Deaths\Accidental_Drug_Related_Deaths_2012-2023.csv"

df = pd.read_csv(file_path)
df.columns

In [22]:
# Remove coordinates
def remove_coordinates(geo_str):
    if isinstance(geo_str, str):
        return geo_str.split('\n')[0].strip()
    return geo_str

df['ResidenceCityGeo'] = df['ResidenceCityGeo'].apply(remove_coordinates)
df['InjuryCityGeo'] = df['InjuryCityGeo'].apply(remove_coordinates)
df['DeathCityGeo'] = df['DeathCityGeo'].apply(remove_coordinates)

In [23]:
# Date parsing
df['Date'] = pd.to_datetime(df['Date'], errors = 'coerce')
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Weekday'] = df['Date'].dt.day_name()

In [24]:
# Creating age groups
bins = [0, 17, 25, 35, 45, 55, 65, 100]
labels = ['<18', '18-25', '26-35', '36-45', '46-55', '56-65', '66+']
df['Age Group'] = pd.cut(df['Age'], bins = bins, labels = labels, right = True)

In [25]:
# Count number of substances present
substance_columns = [
    'Heroin', 'Heroin death certificate (DC)', 'Cocaine', 'Fentanyl',
    'Fentanyl Analogue', 'Oxycodone', 'Oxymorphone', 'Ethanol',
    'Hydrocodone', 'Benzodiazepine', 'Methadone', 'Meth/Amphetamine',
    'Amphet', 'Tramad', 'Hydromorphone', 'Morphine (Not Heroin)',
    'Xylazine', 'Gabapentin', 'Opiate NOS', 'Heroin/Morph/Codeine',
    'Other Opioid', 'Any Opioid', 'Other'
]
df['Num_Substances'] = df[substance_columns].notna().sum(axis=1)

In [26]:
# Clean column names
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace(r'[^\w\s]', '', regex=True)

In [27]:
# Trim whitespace in string fields
categorical_cols = df.select_dtypes(include='object').columns
df[categorical_cols] = df[categorical_cols].apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [29]:
# Filling missing state values (assuming CT since that is the most common)
for col in ['residence_state', 'injury_state', 'death_state']:
    df[col] = df[col].fillna('CT')


In [31]:
# Impute missing demographic values
df['sex'] = df['sex'].fillna('Unknown')
df['race'] = df['race'].fillna('Unknown')

In [33]:
# Drop rows with missing date and remove duplicates
df = df[df['date'].notna()]
df = df.drop_duplicates()

In [34]:
df.to_csv("Cleaned_Drug_Deaths.csv", index=False)